In [1]:
# essential libraries
import os
import math
import numpy as np
import pandas as pd
import datetime as dt

# prediction model
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from PIL import Image

# API, URL, DB
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlencode
from urllib.parse import quote_plus
from urllib.request import urlopen
import sqlite3




# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

#pandas와 같이 plotly에서도 간단한 시각화 지원
import cufflinks as cf
cf.go_offline(connected=True)

# for offline ploting
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
print(px.colors.qualitative.Prism)

['rgb(95, 70, 144)', 'rgb(29, 105, 150)', 'rgb(56, 166, 165)', 'rgb(15, 133, 84)', 'rgb(115, 175, 72)', 'rgb(237, 173, 8)', 'rgb(225, 124, 5)', 'rgb(204, 80, 62)', 'rgb(148, 52, 110)', 'rgb(111, 64, 112)', 'rgb(102, 102, 102)']


In [3]:
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['#636EFA', '#EF553B', 
                       '#00CC96', '#AB63FA',
                       '#FFA15A', '#19D3F3',
                       '#FF6692', '#B6E880',
                       '#FF97FF', '#FECB52','rgb(57,105,172)']

DEFAULT_PLOTLY_COLORS2 = ['rgb(230,210,0)', 'rgb(241,105,19)','rgb(8,104,172)', 'rgb(0,109,44)', 'rgb(131,75,160)']
DEFAULT_PLOTLY_COLORS2_2 = ['rgb(230,210,0)', 'rgb(241,105,19)', 'rgb(241,105,19)', 'rgb(8,104,172)', 'rgb(8,104,172)', 'rgb(0,109,44)','rgb(0,109,44)', 'rgb(131,75,160)', 'rgb(131,75,160)', 'rgb(131,75,160)']

layout_setting = {'xaxis_title':'Date',
                  'yaxis_title':'Number',
                  'font':dict(size=18,color='#60606e',
                              family='Franklin Gothic')}

layout_font = {'font':dict(size=18,color='#60606e',
                           family='Franklin Gothic' )}

In [4]:
path = '../covid'
os.chdir('./data')
file_name_list = os.listdir()

In [5]:
file_name_list

['202003Korea_Population.csv',
 '20200428ncovProvince.csv',
 'avgAgeProvince.jpg',
 'Case.csv',
 'floating_data_2019_03.db',
 'floating_data_2020.db',
 'NaverSearchTrend.csv',
 'PatientInfo.csv',
 'PatientRoute.csv',
 'PatientTrend_20200414_20200427.xlsx',
 'Policy.csv',
 'Region.csv',
 'SearchTrend.csv',
 'SeoulFloating.csv',
 'Time.csv',
 'TimeAge.csv',
 'TimeConfirmed_SCJ.csv',
 'TimeGender.csv',
 'TimeProvince.csv',
 'Weather.csv',
 'WHO-COVID-19-global-data.csv']

In [6]:
case = pd.read_csv("Case.csv")
region = pd.read_csv("Region.csv")
patientinfo = pd.read_csv("PatientInfo.csv")
route = pd.read_csv( 'PatientRoute.csv')
timeProve = pd.read_csv("TimeProvince.csv")
timeAge = pd.read_csv("TimeAge.csv")
timeGender = pd.read_csv("TimeGender.csv")
seoulFloating = pd.read_csv('SeoulFloating.csv')
population = pd.read_csv("202003Korea_Population.csv", encoding='ANSI') # 2020.03 한국 인구수 from 통계청
numProvince = pd.read_csv("20200428ncovProvince.csv", index_col=False) # 2020.04.28 질병관리본부데이터
# http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=354256&contSeq=354256&board_id=140&gubun=BDJ

# Topic 1. 20대 감염원인과 성향 파악

> 기존 EDA를 통해서 연령별 누적 확진자 추이와 사망자 추이를 보면 각 연령별 추이가 꾸준하게 지속되는 것을 확인했습니다. 20대는 꾸준하게 확진자 수가 높았으며 70~80대의 사망률 또한 계속해서 높은 상태를 유지했습니다. 그리하여, 연령대별 특징이나 행동특성이 코로나 감염과의 관계가 있을 것이라고 생각하여 `20대 감염자 어디서 왔는가?!`라는 주제로 분석을 진행해보았습니다.


## 결론 및 요약
코로나 종식 및 예방을 위해서는 가장 많은 감염자 수를 가진 20대 감염자의 수를 줄이는 것이 핵심과제라고 생각했습니다. 그래서 20대를 초점으로 분석을 진행하였고 다음과 같은 결과를 도출하였습니다.

### 1. 20대 감염자 어디서 왔는가?
확연히 들어나는 20대 확진자 수가 많은 원인을 찾아보았습니다. 처음 예상은 EDA를 바탕으로 '해외 유입인구'를 예상하였습니다. 하지만, 대다수가  <span style="color:red">신천지가 대다수</span>를 차지하는 것을 확인하였습니다.

### 2. 과연? 신천지와 해외유입 뿐인가?
신전치를 제외하고도 높은 20대 확진자의 비율을 확인하였습니다. 그 원인을 찾기위해 20대의 행동 특성을 방문경로와 유동인구 측면에서 확인해보았습니다. 그 결과  <span style="color:red">20대는 코로나 바이러스에 대한 위험의식이 낮은 경향이 있으며, 불필요한 방문지역(PC방 등)에 더 많이 방문</span>하는 등 활동 반경이 넓은 특성을 파악했습니다.

### 3. 해결방안
따라서, 가장 우선적으로 신천지와 같은 집단 감염을 막는 것을 막기위해 사회적 거리두기를 적극 실천하고 불필요 방문지역을 줄이도록 해야합니다. 또한, 20대가 많이 방문하는 업종은 사전에 방역 작업과 개인 위생관리를 철저하게 준수하도록하여 확산을 사전에 방지해야합니다. 더 나아가, 20대 스스로 '나는 괜찮다, 젊어서 괜찮다'는 방식의 사고방식을 버리고 코로나에 대한 경각심을 가지고 외출을 자제해야합니다.

---
**이번 포스팅이 유익하고 재밌으셨다면 투표 한번씩 부탁드립니다.**

**분석 과정에 대한 질문과 피드백은 언제든 환영입니다. 감사합니다:)**

이런 결론 내용은 이번에 넣기 애매해서 제외


#### 실제 해외에서 유입되는 코로나 확진자는 눈에 뛰게 높지 않았습니다. 오히려 해외의 경우 이제 막 급격히 확생되는 시점임을 고려했을 때, 해외 입국자가 감염되있을 확률이 높은 것은 사실입니다. 따라서 해외에서 입국하는 내국인 인원들에 대해서 <span style="color:red">감염자 검사, 색출 및 관리</span> 에 집중하여 국내 확산으로 넘어가지 않도록 철처히 차단해야합니다.  
#### 실질적으로 가장 큰 코로나 확산의 원인은 집단 감염인 것으로 밝혀졌습니다. 집단 발생 관련이 약 80%를 차지하고 그 중 약 48%는 신천지 관련으로 발생된 확산이었습니다.[[1]](https://www.cdc.go.kr/board/board.es?mid=a20501000000&bid=0015) 따라서, 국내 코로나 확산 방지를 위해서는 <span style="color:red">많은 인원이 모이는 단체 활동을 피하고 사회적 거리두기를 적극 실천</span>해야합니다. 특히나  코로나 위험성에 대한 경각심이 낮은 <span style="color:red">20대에 대한 집중 관리</span>가 필요하며, 다수의 인원이 모이는 장소는 적극적인 방역 활동과 개인 위생 수칙 준수를 철저히 강조하여 코로나 종식을 향해 전국민이 함께 노력해야합니다.

# 1. 20대 감염자 어디서 왔는가?!

In [7]:
# population.csv 데이터 전처리 과정
cols = list(population.columns)
cols = list(map(lambda x : x[9:] if len(x) > 9 else x, cols))
population.columns = cols
pops = population.iloc[0,3:12]
for i, pop in enumerate(pops.values):
    pops[i] = int(pop.replace(',',""))

# 최신일자 누적 확진자 데이터
recentDay = timeAge['date'].iloc[-1]
recentConfirmed = timeAge[timeAge['date'] == recentDay]['confirmed']
confirmedRatio = np.array(recentConfirmed)*100 / np.array(pops)
C_Ratio_df = pd.Series(confirmedRatio)
C_Ratio_df.index = timeAge['age'].unique()
pops.index = C_Ratio_df.index

In [8]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15, vertical_spacing=0.05,
                   subplot_titles=['<b>연령별 인구수</b>', '<b>누적확진자 비율</b>'])

fig.add_trace(go.Bar(x=pops.index,y=pops.values,
                     marker=dict(color=DEFAULT_PLOTLY_COLORS[1:])), row=1,col=1)

fig.add_trace(go.Bar(x=C_Ratio_df.index,y=C_Ratio_df.values,
                     marker=dict(color=DEFAULT_PLOTLY_COLORS[1:])), row=1, col=2)

fig.update_layout(title='<b>연령별 누적 확진자</b>', **layout_font, showlegend=False)

fig.update_xaxes(title_text='<b>연령 그룹</b>')
fig.update_yaxes(title_text='<b>인구수</b>', row=1, col=1)
fig.update_yaxes(title_text='<b>확진자 수 / 그룹 인구(%)</b>', row=1, col=2)

fig.show()

20대의 인구수에 비해서 40~50대의 인구수가 약 28%만명 더많은 것을 알 수 있습니다. 그럼에도 20대의 확진자 수가 훨씬 많은 것을 알 수 있습니다. 또한, 연령별 인구수에 대한 확진자의 비율을 보면 20대의 확진자 비율이 더욱 극명하게 나타나는 것을 확인할 수 있습니다. 20대의 코로나 확진자가 많은 것의 이유를 알아보겠습니다.

## 1-1) 20대 감염경로 파악, 해외 유입인구
가장 많은 확진자 수를 가진 20대들은 과연 어느 경로를 통해서 유입되는지 확인해보았습니다. 우선 patientinfo 데이터를 사용해서 감염된 경로 및 케이스를 분석해보았습니다. 그 중에서 목표로하는 20대의 확진 케이스가 어떠한지 확인하였습니다. <span style="color:red">해외유입인구</span>가 가장 큰 원인인 것을 확인했습니다.

In [9]:
# 이름 앞부분으로 단축
patientinfo['infection_case'] = patientinfo['infection_case'].astype(str).apply(lambda x : x.split()[0])
infectionCase = patientinfo.pivot_table(index='infection_case',columns='age',
                                        values='patient_id', aggfunc='count')
# 전체 감염 케이스
patientTotal = infectionCase.fillna(0).sum(axis=1)
patientTotal = patientTotal.sort_values(ascending=False)[:5]
# 20대 감염 케이스
patient20s = infectionCase['20s'].dropna()
patient20sTop = patient20s.sort_values(ascending=False)[:5]

In [10]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=['<b>전체 연령</b>', '<b>20 대</b>'])

fig.add_trace(go.Pie(values=patientTotal.values, labels=patientTotal.index, 
                    name='<b>전연령</b>', pull=[.1,0]),row=1,col=1)
fig.add_trace(go.Pie(values=patient20sTop.values, labels=patient20sTop.index, 
                    name='<b>20s</b>', pull=[.1,0]),row=1,col=2,)

fig.update_traces(textinfo="percent+label")
fig.update_layout(title='<b>확진자 감염 케이스</b>', **layout_font, showlegend=True,
                 height=500)
fig.show()

해외에서 유입된 20대 감염자들이 가장 많은 것을 보여주고있습니다. 전체 감염자 비율과는 조금 다르게 해외에서 유입되는 20대의 비율이 상당히 높은 것을 알 수 있습니다. 따라서, 20대의 경우 해외 유입을 통해서 들어온 확진자가 많다고 생각할 수 있습니다. 하지만 환자 정보 데이터의 결측치와 기타 부분이 많아 분석의 한계점과 데이터에 대한 신뢰에 의문을 가졌고 중앙방역대책 본부의 데이터를 통해 다시 분석을 진행하였습니다.

## 1-2) 20대 감염경로 파악, 핵심은 신천지

patientInfo의 부족한 데이터를 보충하기 위해서 중앙방역대책본부의 데이터와 대구광역시 재난안전 대책본부의 발표를 활용하였습니다. 전체 코로나 확진자 중 48.5%가 대구 신천지와 관련된 확진자인 것으로 조사되었으며, 신천지 신도들 중 20대의 비율이 매우 높은 것을 확인하였습니다.

In [11]:
numProvince.index = numProvince['City']
cases = ['Overseas', 'Shincheonji', 'Collective','Patient_contact',
         'Oversea_inflow_related', 'ETC']
confiredSum = numProvince[cases].sum()
confiredSum_df = pd.DataFrame(confiredSum, columns=['num']).reset_index()

In [12]:
fig=px.pie(confiredSum_df, values='num', names='index', hole=.3,
           color_discrete_sequence=px.colors.qualitative.Plotly)
fig.update_traces(textinfo="percent+label")
fig.update_layout(title='<b>확진자 감염 케이스 비율</b>', **layout_font, showlegend=True)
fig.show()

[중앙방역대책본부의 4월28일 현황 발표 데이터](https://www.cdc.go.kr/board/board.es?mid=a20501000000&bid=0015)에 의하면 전체 코로나 확진자 중 48.5%가 대구 신천지와 관련된 확진자인 것으로 조사되었습니다. 

[대구광역시 재난안전대책본부의 3월 6일 발표](https://blog.naver.com/daegu_news/221840129905)에 의하면 신천지 교인 10,914명 중 90% 이상이 진단검사를 받은 것으로 파악되고 이 중 3,617명이 확진판정을 받았습니다. 신천지 교인 3,617명 중 남성이 1,170명(32.3%), 여성은 2,447명(67.7%)로 여성이 2배 이상 많으며, <span style="color:red">20대 1,376명(38.0%)</span>, 50대 663명(18.3%), 40대 496명(13.7%) 순으로, 20대와 40~50대 비율이 높은 것으로 확인되었습니다. 

즉, <span style="color:red">상당수의 20대 확진자가 신천지 교회에서 발생된 것</span>을 확인할 수 있었습니다. 그리하여 대부분의 20대 감염 원인은 신천지에서 유입된 것으로 판단할 수 있었습니다.

# 2. 20대 높은 감염자수 오직 신천지 때문인가?

[3월 23일 중앙방역대책본부장의 정례 브리핑 내용](https://www.youtube.com/watch?v=6BnKcnACgnk)에 의하면 신천지 교인 중 20대가 많은 점도 있지만 <span style="color:red">신천지 교인을 제외하더라도 20대가 많다</span>고 발표하였습니다. 그리하여 여전히 20대 확진자가 왜 많은지에 대한 궁금증은 이어졌고 이를 파악하기위해 유동인구와 방문지역을 중심으로 분석을 진행하였습니다.

## 2-1) 20대의 유동인구 파악 
가장 먼저 생각해볼 수 있는 것이 유동인구라고 생각했습니다. [SKT 빅데이터 허브](https://www.bigdatahub.co.kr/index.do)에서 제공하는 기지국 신호정보를 기반으로 서울시 유동인구를 활용하였습니다. API 연동 부분의 코드는 AccessKey 공개여부 때문에 생략했고, 저장된 DB에서 불러오는 부분만 첨부하였습니다.

코로나 전과 후의 유동인구를 파악하기위해서 공개된 데이터 최초인 2019년 3월에서 시작해서 가장 최근 데이터인 2020년 2월까지의 데이터를 활용했습니다.

In [13]:
# 2019년 3월 ~ 2019년 5월까지 유동인구 
# SKT 빅데이터 허브 API 데이터 
base_url = "https://api.bigdatahub.co.kr/v1/datahub/datasets/search.json"

DB = pd.DataFrame()
for mon in [1002254,1002255,1002256]: # 3,4,5 월
    for i in range(1,3001):
        params = urlencode({'TDCAccessKey' : 'ce05a26bfe95d56be0f709b10e71d5a6c19cf99a97213ea57fc6252c1af322a9',
                 'pid' : mon, 
                 '$page' : i, '$count' : 3000,
                 '$select' : '*',
                 '$where' : '시간(1시간단위) = 18'})
        data = requests.get(url=base_url, params=params).json()
        if data['entry'] == [] : break
        db = pd.DataFrame(data['entry'])
        DB = pd.concat([DB, db])
        
DB.columns = seoulFloating.columns
DB['fp_num'] = DB['fp_num'].astype(int)

sql = sqlite3.connect(r"floating_data_2019_03.db")
DB.to_sql('moving1903',sql, if_exists='replace')

In [14]:
sql = sqlite3.connect(r"floating_data_2019_03.db")
floating_02 = pd.read_sql("SELECT * FROM moving1903", sql)
floating_02 = floating_02.drop(['index'], axis=1, errors='ignore')

fp_age = pd.pivot_table(floating_02, index='date',columns='birth_year',values='fp_num')
fp_age.index = list(map(lambda x : x[:4] + '-' + x[4:6] + '-' + x[6:], fp_age.index))
fig = go.Figure()
for col in fp_age.columns:
    fig.add_trace(go.Scatter(x=fp_age.index, y=fp_age[col],mode='lines+markers', name=col) )

fig.update_layout(title='<b>2019년 3~5월 유동인구</b>', **layout_setting)
fig.show()

20대의 유동인구가 많을 것이라고 예상했던 것과는 다르게 <span style="color:red">30~40대의 유동인구가 가장 많았습니다.</span> 그 다음으로 20대와 50대가 유동인구 측면에서 유사한 것을 확인할 수 있었습니다. 

전반적으로 평일은 회사나 학원 등 일상생활을 진행하기에 유동인구가 주말에 비해 높으면서 일정하게 유지되는 것을 확인할 수 있습니다. 이후, 주말은 토요일과 일요일 순서로 점차 유동인구가 감소하는 것을 확인할 수 있습니다.

20대 행동 패턴으로 확인할 수 있는 약간의 특징으로는 우리나라의 `불금` 성향을 따라 금요일 20대의 유동인구가 늘어나는 것을 확인할 수 있습니다. 또한, 다른 연령대에 비해서 상대적으로 토요일 유동인구가 많은 것 또한 확인할 수 있습니다.

In [15]:
# base_url = "https://api.bigdatahub.co.kr/v1/datahub/datasets/search.json"

# DB_2020 = pd.DataFrame()
# for mon in [1002286,1002291]: # 1월 api pid, 2월 api pid
#     for i in range(1,3001):
#         params = urlencode({'TDCAccessKey' : 'ce05a26bfe95d56be0f709b10e71d5a6c19cf99a97213ea57fc6252c1af322a9',
#                  'pid' : mon, 
#                  '$page' : i, '$count' : 3000,
#                  '$select' : '*',
#                  '$where' : '시간(1시간단위) = 18'})
#         data = requests.get(url=base_url, params=params).json()
#         if data['entry'] == [] : break
#         db = pd.DataFrame(data['entry'])
#         DB_2020 = pd.concat([DB_2020, db])
        
# DB_2020.columns = seoulFloating.columns
# DB_2020['fp_num'] = DB_2020['fp_num'].astype(int

# sql = sqlite3.connect(r"floating_data_2020.db")
# DB_2020.to_sql('moving',sql, if_exists='replace')

In [16]:
sql = sqlite3.connect(r"floating_data_2020.db")
floating = pd.read_sql("SELECT * FROM moving", sql)
floating = floating.drop(['index'], axis=1, errors='ignore')

fp_age = pd.pivot_table(floating, index='date',columns='birth_year',values='fp_num')
fp_age.loc['20200223'] = fp_age.loc['20200223']/2 # 이상치 처리
fp_age.index = list(map(lambda x : x[:4] + '-' + x[4:6] + '-' + x[6:], fp_age.index))
fig = go.Figure()
for col in fp_age.columns:
    fig.add_trace(go.Scatter(x=fp_age.index, y=fp_age[col],mode='lines+markers', name=col) )

fig.update_layout(title='<b>2020년 1~2월 유동인구</b>', **layout_setting)
fig.show()

3월 데이터가 아직 공개되지 않은 관계로 2020년 1월~2월 까지의 데이터를 API를 통해 가져왔습니다. 1월 25일 근처로 깊은 유동인구의 하락은 설날로 인한 지방으로 전체적인 인구가 이동하면서 발생된 감소이므로 분석에서 제외하고 그래프를 보시면 되겠습니다. 

설날 이후 중국에서 넘어온 코로나에 대한 경각심이 시작된 시점입니다. 1월 31일을 기점으로 60, 70대의 유동인구가 급격하게 감소하였으며, 그 추세를 이어가고 있습니다. 또한, 오히려 20대의 전반적인 유동인구의 수가 50대와 더 차이가 나는 것을 볼 수 있었습니다. 

5천만 인구인 대한민국에서 서울시의 유동인구를 대표하기에 부족한점, 동일 기간의 데이터가 아닌점, 대학과 학교가 개학하지 않고 온라인 수업을 진행한 것, 코로나에 민감해진 3월~4월의 유동인구 데이터가 없는 점 등의 한계가 많았습니다. 

결론적으로 <span style="color:red">유동인구를 통한 20대의 코로나 확진자 관계는 발견되지 않았습니다.</span> 하지만, 한국의 정책에서 1월 28일 Level 3 질병 경보 이후의 고연령층의 유동인구 변화, 금요일과 주말의 유동인구의 변화, 3월 이후의 유동인구의 변화 등 더 유의미한 의미를 찾아낼 수 있을 것으로 예상되었습니다. 이후 다음 추가적인 연구 및 분석을 진행해볼 예정입니다.

## 2-2) 20대 확진자들의 이동경로 파악

유동인구 측면에서 20대의 움직임이 많지 않았습니다. 하지만 금요일 저녁부터 토요일까지의 유동인구가 다른 연령대에 비해 훨씬 많은 것을 파악했습니다. 사회활동이 활발한 20대의 특징을 확연히 볼 수 있었습니다. 이를 바탕으로 실제 확진자들을 기준으로 20대가 어떻게 움직였는지를 확인하기 위한 분석을 진행했습니다. 환자의 이동경로를 나타낸 Route 데이터셋과 환자 정보가 전반적으로 나타나있는 PatientInfo 데이터를 환자의 아이디 번호를 기준으로 합병한 데이터를 활용하였습니다. 

In [17]:
patientinfo['infection_case'] = patientinfo['infection_case'].astype(str).apply(lambda x : x.split()[0])
infectionCase = patientinfo.pivot_table(index='infection_case',columns='age',values='patient_id', aggfunc='count')
mergeTemp = pd.merge(route, patientinfo,on='patient_id')
pinfoANDroute = mergeTemp.pivot_table(index='type', columns=['age'], values='patient_id', aggfunc='count')
pinfoANDroute.index = list(map(lambda x : x.split('_')[0], pinfoANDroute.index))

In [18]:
pinfoANDroute = pd.merge(route, patientinfo,on='patient_id').pivot_table(index='type', columns=['age'], values='patient_id', aggfunc='count')
pinfoANDroute.index = list(map(lambda x : x.split('_')[0], pinfoANDroute.index))
target = ["20s","30s","40s","50s"]

fig = make_subplots(2,2, horizontal_spacing=0.1,
                   vertical_spacing=0.1, subplot_titles=[f"{x}" for x in target])

for i, column in enumerate(target):
    row, col = i//2 + 1, i%2 + 1
    infectionRoute = pinfoANDroute[column].dropna()
    fig.add_trace(go.Bar(x=infectionRoute.values, y=infectionRoute.index,
                         name=column, orientation='h'),row=row, col=col)
    
fig.update_layout(title="<b>연령별 확진자 방문지역 Case</b>",
                  margin=dict(l=20, r=20, t=100, b=20),
                  height=600, width=1100, **layout_font)    
fig.show()

환자들의 이동경로가 결국 마지막은 병원으로 이동해서 확진을 받습니다. etc는 파악이 불가능하고 모든 집단에서 가장 높은 비율을 가지고 있다는 점을 고려하여, 두 경로를 분석에서 제외하였습니다. 

생활하는 과정에 불필요한 방문지와 필요 방문지역을 다음과 같이 구분하였습니다.

불필요 방문 업종 : 부동산, 미용, 제과점, 헬스장, 바, 숙박업소, pc방, 카페, 교회, 가게, 레스토랑

필요 방문 업종 : 관공서, 은행, 학업관련, 약국, 대중교통 등 

전체 확진자들 보다는 20대와 가장 유사한 유동인구 수를 보여주었던 50대와의 비교를 통해 분석을 진행했습니다.

In [19]:
pinfoANDroute.drop(['hospital','etc'], inplace=True, errors='ignore')

not_necessary = ['real', 'beauty', 'bakery', 'gym', 'bar', 'lodging',
                 'pc', 'cafe', 'church', 'store', 'restaurant']

target = ["20s", "50s"]

fig = make_subplots(1,2, horizontal_spacing=0.15, subplot_titles=[f"{x}" for x in target])

for i, column in enumerate(target):
    row, col = i//2 + 1, i%2 + 1
    infectionRoute = pinfoANDroute.loc[:,column].fillna(0)
    infectionRoute = infectionRoute / sum(infectionRoute)
    infectionRoute = infectionRoute[not_necessary]
    fig.add_trace(go.Bar(x=infectionRoute.values, y=infectionRoute.index,
                         name=column, orientation='h'),row=row, col=col)
    
fig.update_layout(title="<b>20대와 50대의 불필요 방문지역</b>",
                  margin=dict(l=20, r=20, t=100, b=20),
                  height=600, width=800, **layout_font)    

fig.update_xaxes(title_text="<b>방문 비율</b>")
fig.update_yaxes(title_text="<b>방문 지역</b>", row=1, col=1)

fig.show()

필수 방문지가 아닌 곳, `not necessary`에 방문한 비율을 구하여 비교하였습니다. 공통적으로 많이 방문하는 store과 church는 비슷한 양상을 보여주고 있습니다. 하지만, 20대는 음식점, pc방, cafe, bar 등에서 훨씬 많은 방문비율을 확인할 수 있었습니다. 

더욱 자세하게 공공장소(public), 일상생활(life), 엔터(entertainment)부분으로 나누어서 보았습니다. 20대가 엔터부분에 방문하는 비율이 가장 높은 것을 다시한번 확인할 수 있습니다.

In [20]:
public = ['administrative','airport','bank','post','real','public','academy', 'school', 'university']
life = ['bakery','gas','store']
enter = ['bar','beauty','cafe','lodging','pc','restaurant','church','gym']
medical = ['hospital','pharmacy']
cluster = [public, life, enter]

infectionRoute = pinfoANDroute.loc[:,].fillna(0)
infectionRoute = infectionRoute / infectionRoute.sum()

def clustering(df, col):
    target = df.fillna(0)[col]
    value = []
    for part in cluster:
        s = target[part].sum()
        value.append(s)
    return value
    
container = []
for col in infectionRoute.columns:
    container.append(clustering(infectionRoute, col))

categories = ['public','life','entertainment']

fig = go.Figure()

for col in ['20s','30s','40s','50s','60s']:    
    fig.add_trace(go.Scatterpolar(r=clustering(infectionRoute, col), 
                                  theta= categories, fill='toself',
                                  
                                  line=dict(width=0.5), name=col))

fig.update_layout(polar=dict(radialaxis=dict(visible=False, type='log')))
fig.update_layout(title="<b>방문 군집 비교</b>", **layout_font)    

    
fig.show()

이를 통해서 불필요 지역 방문이 높은 <span style="color:red">20대는 활동 반경이 넓고 활발한 것을 고려하여 코로나에 더 쉽게 노출될 수 있을 것이라 판단</span>하였습니다.

### 외부 자료 및 보도

위의 결과 외에도, 서울대학교 보건대학원 유명순 교수팀이 지난달 25~27일간 한국리서치에 의뢰해 진행한 [‘코로나19국민 인식조사’를 분석결과](https://news.joins.com/article/23751923)에 따르면, <span style="color:red">20대의 코로나 바이러스에 대한 인식이 매우 낮다는 결과</span>가 나왔습니다. 

그 뿐만 아니라, [4월 27일 질병관리본부에서 발표한 보도자료](https://www.cdc.go.kr/board/board.es?mid=a20501000000&bid=0015&list_no=367014&act=view)에 따르면 대구 확진자가 부산 클럽, 주점 및 숙박업소 등에 방문하며 143명의 접촉자가 발생했습니다.  

또한, 강남과 홍대 클럽이 개장하며 길게 줄서있는 모습, [자가격리 중 매일 스타벅스 간 20대 서초구 36번 확진자](https://www.nocutnews.co.kr/news/5325329) 등을 보았을 때, 20대들의 코로나에 대한 경각심과 인식이 부족한 경향이있습니다.

### 결론 및 요약
20대 확진자의 비율은 신천지 집단 감염 발생으로 인한 것입니다. 하지만, 신천지 외의 20대 확진자의 비율은 여전히 타 연령대에 비해 높습니다. 그 이유는 20대의 코로나에 대한 경각심 부족으로 불필요 방문지역에 다수 방문하는 것을 원인으로 판단하였습니다. 
따라서, 코로나 확산 방지 및 종식을 위해서는 사회적 활동 반경이 넓고 움직임이 많은 20대의 적극적인 사회적 거리두기와 외출 및 불필요 지역 방문 횟수를 자제해야합니다.

코로나 확산 방지 및 종식을 위해서는 가장 우선적으로 신천지와 같은 집단 감염을 막는 것을 막아야합니다. 그러므로 사회적 거리두기를 적극 실천하고 불필요 방문지역을 줄이도록 해야합니다. 또한, 20대가 많이 방문하는 업종은 사전에 방역 작업과 개인 위생관리를 철저하게 준수하도록하여 확산을 사전에 방지해야합니다. 더 나아가, 20대 스스로 '나는 괜찮다, 젊어서 괜찮다'는 방식의 사고방식을 버리고 코로나에 대한 경각심을 가지고 외출을 자제해야합니다.

# 최종 결론

코로나 종식 및 예방을 위해서는 <span style="color:red">해외 유입에 의한 확진자를 차단</span>해야합니다. 현재 입국자에 대한 검사 및 2주 자가격리 등 많은 노력이 진행되고 있습니다. 하지만, 그럼에도 유의사항을 잘 따르지 않는 일부 인원에 의해서 신천지와 같은 큰 집단 감염이 발생될 수 있다는 사실을 잊지 말아야합니다. 따라서, 코로나에 대한 경각심과 인식을 잘 심어주어야하며, 특히나 가장 안일하게 생각하는 20대의 인식 변화를 이끌어야 할 것입니다. 
또한, <span style="color:red">20대의 행동 패턴 및 방문 경로를 바탕으로 감염 위험이 있는 업종은 특히나 더욱 신경써서 사회적 거리두기, 마스크 착용, 손세정제와 손씻기 등을 더욱 권장</span>하도록 해야합니다.